In [1]:
import numpy as np
import notebooks.scripts.geometrical_objects as geos
import plotly.express as px
import pandas as pd


In [2]:
disk = geos.Disk(5)

In [3]:
def radius_normalizer(disk):
	def _normalizer(r, alpha):
		return np.stack([ np.sqrt(disk.radius*r), alpha])
	return _normalizer

def angle_normalizer(disk):
    def _normalizer(r, alpha):
        rev_r = disk.radius - r
        rev_alpha = 2*np.pi - alpha
        is_oob = alpha*disk.radius/r > 2*np.pi
        r2 = r.copy()
        r2[is_oob] = rev_r[is_oob]
        alpha2 = alpha.copy()
        alpha2[is_oob] = rev_alpha[is_oob]
        alpha2 = alpha2*disk.radius*1/r2
        return np.stack([r2, alpha2])
    return _normalizer

In [10]:
num_samples = 10_000
r = disk.radius*np.random.rand(num_samples)
alpha = 2*np.pi*np.random.rand(num_samples)

In [11]:
r_radius_norm, alpha_radius_norm = radius_normalizer(disk)(r, alpha).tolist()
r_angle_norm, alpha_angle_norm = angle_normalizer(disk)(r, alpha).tolist()

In [15]:
x1, x2 = disk(r, alpha).T.tolist()
samples = pd.DataFrame({
	'radius': r,
	'angle': alpha,
	'x': x1,
    'y': x2
})

x1, x2 = disk(r_radius_norm, alpha_radius_norm).T.tolist()
samples_radius_norm = pd.DataFrame({
    'radius': r_radius_norm,
    'angle': alpha_radius_norm,
    'x': x1,
    'y': x2
})

x1, x2 = disk(r_angle_norm, alpha_angle_norm).T.tolist()
samples_angle_norm = pd.DataFrame({
    'radius': r_angle_norm,
    'angle': alpha_angle_norm,
    'x': x1,
    'y': x2
})

samples['Type'] = 'Original'
samples_radius_norm['Type'] = 'Radius Normalized'
samples_angle_norm['Type'] = 'Angular Normalized'

df = pd.concat([samples, samples_radius_norm, samples_angle_norm])

In [19]:
fig = px.scatter(df, x='x', y='y', color ='Type', width = 1300, height=500, color_discrete_sequence=['#CC7777', '#77CC77', '#7777CC'], opacity = 0.3, facet_col='Type')
fig.update_layout(showlegend = False)
fig.write_image("disk_sampling.png")


In [18]:
fig = px.scatter(df, x='radius', y='angle', color ='Type', width = 1300, height=500, color_discrete_sequence=['#CC7777', '#77CC77', '#7777CC'], opacity = 0.3, facet_col='Type')
fig.update_layout(showlegend = False)
fig.write_image("disk_domain_sampling.png")
